In [ ]:
from transformers import pipeline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import tqdm

In [ ]:
device = 'mps'

In [ ]:
def batch_encoding_to_device(batch):
    return {k: v.to(device) for k, v in batch.items()}

## 3. Fine-Tuning

### 3.1 Processing data

In [ ]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
checkpoint = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
model.to(device);

In [ ]:
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]

In [ ]:
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors='pt').to(device)

In [ ]:
batch['labels'] = torch.tensor([1, 1]).to(device)

In [ ]:
#optimizer = AdamW(model.parameters())
optimizer = torch.optim.AdamW(model.parameters())

In [ ]:
loss = model(**batch).loss

In [ ]:
loss.backward()

In [ ]:
optimizer.step()

In [ ]:
from datasets import load_dataset

In [ ]:
raw_datasets = load_dataset('glue', 'mrpc')

In [ ]:
raw_datasets

In [ ]:
raw_train_dataset = raw_datasets['train']

In [ ]:
raw_train_dataset[0]

In [ ]:
raw_train_dataset.features

In [ ]:
tokenized_sentences_1 = tokenizer(raw_train_dataset['sentence1'])
tokenized_sentences_2 = tokenizer(raw_train_dataset['sentence2'])

In [ ]:
tokenized_sentences_1.keys()

In [ ]:
token_lengths = []
for i, j in zip(tokenized_sentences_1['input_ids'], tokenized_sentences_2['input_ids']):
    token_lengths.append([len(i), len(j)])

In [ ]:
token_lengths = np.array(token_lengths)

In [ ]:
plt.hist(token_lengths[:, 0], alpha=0.5)
plt.hist(token_lengths[:, 1], alpha=0.5)

In [ ]:
inputs = tokenizer('This is the first sentence.', 'This is the second one.')
inputs1 = tokenizer('This is the first sentence.')
inputs2 = tokenizer('This is the second one.')

In [ ]:
# token_type_ids --> which tokens belong to which sentences
inputs

In [ ]:
inputs1

In [ ]:
inputs2

In [ ]:
decoded = tokenizer.convert_ids_to_tokens(inputs['input_ids'])

In [ ]:
print(decoded)

In [ ]:
tokenized_dataset = tokenizer(
    raw_train_dataset['sentence1'],
    raw_train_dataset['sentence2'],
    padding=True,
    truncation=True,
    return_tensors='pt'
)

In [ ]:
# no padding in function since it'll be used on batches and batches will have different lengths
def tokenize_function(example):
    return tokenizer(example['sentence1'], example['sentence2'], truncation=True)

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

In [ ]:
tokenized_datasets

In [ ]:
from transformers import DataCollatorWithPadding

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
data_collator

In [ ]:
samples = tokenized_datasets['train'][:8]

In [ ]:
samples = {k: v for k, v in samples.items() if k not in ['idx', 'sentence1', 'sentence2']}

In [ ]:
[len(x) for x in samples['input_ids']]

In [ ]:
batch = data_collator(samples)

In [ ]:
{k: v.shape for k, v in batch.items()}

### 3.2 Finte-tuning a model with Trainer API

In [ ]:
from datasets import load_dataset

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding

In [ ]:
raw_datasets = load_dataset('glue', 'mrpc')

In [ ]:
checkpoint = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
def tokenize_function(example):
    return tokenizer(example['sentence1'], example['sentence2'], truncation=True)

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import TrainingArguments

In [ ]:
training_args = TrainingArguments(
    'test-trainer',
    #eval_strategy='epoch', # eval every epoch
    #num_train_epochs=3
)

In [ ]:
training_args.per_device_train_batch_size, training_args.per_device_eval_batch_size

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
model.to(device);

In [ ]:
from transformers import Trainer

In [ ]:
# handles putting data on device
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    #data_collator=data_collator, # uses data collator with padding by default
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

In [ ]:
predictions = trainer.predict(tokenized_datasets['validation'])

In [ ]:
predictions.predictions.shape, predictions.label_ids.shape

In [ ]:
predictions.metrics

In [ ]:
preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:
preds.shape

In [ ]:
import evaluate

In [ ]:
metric = evaluate.load('glue', 'mrpc')

In [ ]:
metric.compute(predictions=preds, references=predictions.label_ids)

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load('glue', 'mrpc')
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments('test-trainer', evaluation_strategy='epoch')
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [ ]:
# trainer automatically attaches model to gpu is available
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
predictions = trainer.predict(tokenized_datasets['validation'])

In [ ]:
predictions.metrics

### 3.3 A full training

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

In [ ]:
raw_datasets = load_dataset('glue', 'mrpc')
checkpoint = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
def tokenize_function(example):
    return tokenizer(example['sentence1'], example['sentence2'], truncation=True)

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(['sentence1', 'sentence2', 'idx'])
tokenized_datasets = tokenized_datasets.rename_column('label', 'labels')
tokenized_datasets.set_format('torch')

In [ ]:
tokenized_datasets['train'].column_names

In [ ]:
import torch
from torch.utils.data import DataLoader

In [ ]:
train_dataloader = DataLoader(
    tokenized_datasets['train'], shuffle=True, batch_size=8, collate_fn=data_collator
)

In [ ]:
eval_dataloader = DataLoader(
    tokenized_datasets['validation'], shuffle=True, batch_size=8, collate_fn=data_collator
)

In [ ]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [ ]:
model.to(device);

In [ ]:
batch_device = batch_encoding_to_device(batch)

In [ ]:
outputs = model(**batch_device)

In [ ]:
outputs.loss, outputs.logits.shape

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

In [ ]:
from transformers import get_scheduler

In [ ]:
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    'linear',
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [ ]:
num_training_steps

In [ ]:
progress_bar = tqdm.auto.tqdm(range(num_training_steps))

In [ ]:
lrs = []
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch_device = batch_encoding_to_device(batch)
        outputs = model(**batch_device)
        loss = outputs.loss
        loss.backward()

        lrs.append(lr_scheduler.get_lr())
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

In [ ]:
import evaluate

In [ ]:
metric = evaluate.load('glue', 'mrpc')

In [ ]:
model.eval()
for batch in tqdm.tqdm(eval_dataloader):
    batch_device = batch_encoding_to_device(batch)
    with torch.no_grad():
        outputs = model(**batch_device)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch['labels'])

metric.compute()

### Accelerate (distributed GPU training; can only use the one MPS GPU)

In [ ]:
import torch

In [ ]:
from accelerate import Accelerator
from transformers import AutoModelForSequenceClassification, get_scheduler

In [ ]:
accelerator = Accelerator()

In [ ]:
accelerator

In [ ]:
accelerator.device

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)

In [ ]:
model.to(device);

In [ ]:
train_dataloader, eval_dataloader, model, optimizer = accelerator.prepare(
    train_dataloader, eval_dataloader, model, optimizer
)

In [ ]:
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
  "linear",
  optimizer=optimizer,
  num_warmup_steps=0,
  num_training_steps=num_training_steps
)

In [ ]:
progress_bar = tqdm.auto.tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
  for batch in train_dataloader:
      outputs = model(**batch)
      loss = outputs.loss
      accelerator.backward(loss)

      optimizer.step()
      lr_scheduler.step()
      optimizer.zero_grad()
      progress_bar.update(1)

On command line:
```
accelerate config
accelerate launch train.py
```